# 얼굴 인식, 점찍기

# 얼굴 인식, 점찍기

In [9]:
import mediapipe as mp
import cv2

# 얼굴에서 특징점을 찾을 수 있는 기능 불러오기
mp_face_mesh = mp.solutions.face_mesh
# 찾은 특징점 표현하기
mp_drawing = mp.solutions.drawing_utils
# 특징점 그리기 속성
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color= (0,0,255))
# 얼굴에서 특징점 찾기 속성
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence = 0.5, # 얼굴 검출 확률(자체판단) 50% 이상인 것들만 출력하기
    min_tracking_confidence = 0.5 # 특징점 검출 확률(자체판단) 50% 이상인 것들만 출력하기
)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 카메라에서 불러온 사진에서 얼굴 틀징점 찾기
    results = face_mesh.process(img)

    # 얼굴의 특징점(478)을 감지했을 때 실행하겠다
    if results.multi_face_landmarks is not None:
        # print(results.multi_face_landmarks)
        for res in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(img, res, mp_face_mesh.FACEMESH_TESSELATION)
    
    cv2.imshow('video', img)
    if cv2.waitKey(33) == 49:
        break

cap.release()
cv2.destroyAllWindows()

# 코에 점 찍기

# 코에 점 찍기

In [17]:
import mediapipe as mp
import cv2

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color= (0,0,255))
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence = 0.5, 
    min_tracking_confidence = 0.5 
)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    results = face_mesh.process(img)

    if results.multi_face_landmarks is not None:
        face_landmarks = results.multi_face_landmarks[0]
        # 코의 좌표 찾기
        x = int(face_landmarks.landmark[4].x * img.shape[1])
        y = int(face_landmarks.landmark[4].y * img.shape[0])
        # print(x,y)
        cv2.circle(img, (x,y), 20, (0,0,255), cv2.FILLED)
    
    cv2.imshow('video', img)
    if cv2.waitKey(33) == 49:
        break

cap.release()
cv2.destroyAllWindows()

# 호랑이 가면 합성하기

# 호랑이 가면 합성하기

In [23]:
import mediapipe as mp
import cv2

tiger = cv2.imread('images/psy_img/tiger_mask.png')
mask2gray = cv2.cvtColor(tiger , cv2.COLOR_BGR2GRAY)
_, mask_b = cv2.threshold(mask2gray, 200, 255, cv2.THRESH_BINARY)
mask_b_inv = cv2.bitwise_not(mask_b)

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color= (0,0,255))
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence = 0.5, 
    min_tracking_confidence = 0.5 
)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    results = face_mesh.process(img)

    if results.multi_face_landmarks is not None:
        face_landmarks = results.multi_face_landmarks[0]
        x = int(face_landmarks.landmark[4].x * img.shape[1])
        y = int(face_landmarks.landmark[4].y * img.shape[0])

        # 이미지를 집어넣을 중심 좌표(코의 위치) > x,y
        # 마스크 이미지의 크기 > 123,132(tiger.shape)

        # 외곽으로 가면 이미지 범위를 벗어나서 오류남으로 try, except
        try:
            sub_img = img[y-61:y+62, x-66:x+66]
            img_bg = cv2.bitwise_and(sub_img, sub_img, mask = mask_b)
            img_fg = cv2.bitwise_and(tiger,tiger, mask = mask_b_inv)
            bg_fg = cv2.add(img_bg,img_fg)
            img[y-61:y+62, x-66:x+66] = bg_fg
        except:
            pass
    
    cv2.imshow('video', img)
    if cv2.waitKey(33) == 49:
        break

cap.release()
cv2.destroyAllWindows()

# resize

# resize

In [52]:
import mediapipe as mp
import cv2

# 크기를 조정할 전역 변수
TIGER_WIDTH = 150  # 이미지의 가로 크기
TIGER_HEIGHT = 150  # 이미지의 세로 크기

# 위치를 동적으로 계산
TIGER_OFFSET_X = -(TIGER_WIDTH // 2)
TIGER_OFFSET_Y = -(TIGER_HEIGHT // 2)

# tiger = cv2.imread('images/psy_img/botchi.jpg')
tiger = cv2.imread('images/psy_img/anaya.jpg')
# 이미지 사이즈 조정
tiger = cv2.resize(tiger, (TIGER_WIDTH, TIGER_HEIGHT))
mask2gray = cv2.cvtColor(tiger, cv2.COLOR_BGR2GRAY)
_, mask_b = cv2.threshold(mask2gray, 240, 255, cv2.THRESH_BINARY)
mask_b_inv = cv2.bitwise_not(mask_b)

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, color=(0, 0, 255))
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    results = face_mesh.process(img)

    if results.multi_face_landmarks is not None:
        face_landmarks = results.multi_face_landmarks[0]
        x = int(face_landmarks.landmark[4].x * img.shape[1])
        y = int(face_landmarks.landmark[4].y * img.shape[0])

        try:
            sub_img = img[y + TIGER_OFFSET_Y:y + TIGER_OFFSET_Y + TIGER_HEIGHT,
                          x + TIGER_OFFSET_X:x + TIGER_OFFSET_X + TIGER_WIDTH]
            img_bg = cv2.bitwise_and(sub_img, sub_img, mask=mask_b)
            img_fg = cv2.bitwise_and(tiger, tiger, mask=mask_b_inv)
            bg_fg = cv2.add(img_bg, img_fg)
            img[y + TIGER_OFFSET_Y:y + TIGER_OFFSET_Y + TIGER_HEIGHT,
                x + TIGER_OFFSET_X:x + TIGER_OFFSET_X + TIGER_WIDTH] = bg_fg
        except:
            pass

    cv2.imshow('video', img)
    if cv2.waitKey(33) == 49:
        break

cap.release()
cv2.destroyAllWindows()
